<a href="https://colab.research.google.com/github/stanlee321/COLAB_object_detection_finetune/blob/master/fine_tune_object_detection_COLAB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-tune Object Detection on Google Colab

First, mount your Google Drive 

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In your drive, I expect that you have the folder "object_detection" created inside of your "Colab Notebooks" folder.

Navigate to this folder with %cd command.

In [0]:
%cd "/content/drive/My Drive/Colab Notebooks/object_detection"

/content/drive/My Drive/Colab Notebooks/object_detection


I move this  .ipynb to this folder, previously I have the folder "models/" also.

I check this with the %ls command.

In [0]:
%ls

fine_tune_object_detection.ipynb  models/


## Clone and install.

Clone the models repo into this folder.

In this step we clone the models repo for retrain the object detection model, also we install some tools like coco_api and also we create the ENVIROMENT PATHS for the object detection.

In [0]:
!git clone --quiet https://github.com/tensorflow/models.git

In [0]:
# Install some tools

!apt-get install -qq protobuf-compiler python-tk
!pip install -q Cython contextlib2 pillow lxml matplotlib PyDrive
!pip install -q pycocotools


In [0]:
# Go to models/research folder

%cd models/research/

/content/drive/My Drive/Colab Notebooks/object_detection/models/research


In [0]:
# Install object detection api
!protoc object_detection/protos/*.proto --python_out=.

In [0]:
# Verify the folder models/research

%pwd

'/content/drive/My Drive/Colab Notebooks/object_detection/models/research'

In [0]:
# Set env paths for object detection api

import os
os.environ['PYTHONPATH'] += ':/content/drive/My Drive/Colab Notebooks/object_detection/models/research/:/content/drive/My Drive/Colab Notebooks/object_detection/models/research/slim/'

In [0]:
# Verify tensorflow object detection API installation 

!python object_detection/builders/model_builder_test.py


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

............s...
----------------------------------------------------------------------
Ran 16 tests in 0.059s

OK (skipped=1)


## 3. Download the pre-trained model

We must to download our pretrained model that we want to fine-tune.

In this case I choose the "ssd_mobilenet_v2_quantized_300x300_coco_2019_01_03" from the [models zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md).


I'm going to download the model into the `models/research/object_detection`, so I go to the `object_detection/` folder with `%cd`.

In [0]:
#  Go to object detection folder 

%cd object_detection/

/content/drive/My Drive/Colab Notebooks/object_detection/models/research/object_detection


Change the name of your model in MODEL_NAME if you want to test with another model.

In [0]:
# Model downloaded from
# https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md


MODEL_NAME = 'ssd_mobilenet_v2_quantized_300x300_coco_2019_01_03' # change this name if you want to test with other model
MODEL_FILE = MODEL_NAME + '.tar.gz'
DOWNLOAD_BASE = "http://download.tensorflow.org/models/object_detection/"



Download and untar the file.  In line 15 get the "download link" for the choosen model from the models zoo.

In [0]:
import os
import shutil
import glob
import urllib
import tarfile

""" THIS URLLIB gets ERROR
if not (os.path.exists(MODEL_FILE)):
  opener = urllib.URLopener()
  opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)
"""

# SO I manually download the model with !wget

!wget "http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v2_quantized_300x300_coco_2019_01_03.tar.gz"
  
# Untar the .tar.gz file

tar = tarfile.open(MODEL_FILE)
tar.extractall()
tar.close()

--2019-05-05 00:22:38--  http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v2_quantized_300x300_coco_2019_01_03.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 74.125.141.128, 2607:f8b0:400c:c06::80
Connecting to download.tensorflow.org (download.tensorflow.org)|74.125.141.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 144806142 (138M) [application/x-tar]
Saving to: ‘ssd_mobilenet_v2_quantized_300x300_coco_2019_01_03.tar.gz.2’

ssd_mobilenet_v2_qu 100%[===================>] 138.10M  74.7MB/s    in 1.8s    

2019-05-05 00:22:40 (74.7 MB/s) - ‘ssd_mobilenet_v2_quantized_300x300_coco_2019_01_03.tar.gz.2’ saved [144806142/144806142]



In [0]:
# CHeck if model folder was downloaded and untar.
%ls

anchor_generators/
box_coders/
builders/
CONTRIBUTING.md
core/
data/
data_decoders/
dataset_tools/
dockerfiles/
eval_util.py
eval_util_test.py
exporter.py
exporter_test.py
export_inference_graph.py
export_tflite_ssd_graph_lib.py
export_tflite_ssd_graph_lib_test.py
export_tflite_ssd_graph.py
g3doc/
inference/
__init__.py
inputs.py
inputs_test.py
legacy/
matchers/
meta_architectures/
metrics/
model_hparams.py
model_lib.py
model_lib_test.py
model_main.py
models/
model_tpu_main.py
object_detection_tutorial.ipynb
predictors/
protos/
__pycache__/
README.md
samples/
ssd_mobilenet_v2_quantized_300x300_coco_2019_01_03/
ssd_mobilenet_v2_quantized_300x300_coco_2019_01_03.tar.gz
ssd_mobilenet_v2_quantized_300x300_coco_2019_01_03.tar.gz.1
test_ckpt/
test_data/
test_images/
training/
utils/


In [0]:
%ls ssd_mobilenet_v2_quantized_300x300_coco_2019_01_03

model.ckpt.data-00000-of-00001  model.ckpt.meta  tflite_graph.pb
model.ckpt.index                pipeline.config  tflite_graph.pbtxt


Yes, it was downloded and untar with the name `ssd_mobilenet_v2_quantized_300x300_coco_2019_01_03/`

In [0]:
# Check the folder where we are working on.

%pwd

'/content/drive/My Drive/Colab Notebooks/object_detection/models/research/object_detection'

##  Check dataset.

Check our dataset into the folder:

`/content/drive/My Drive/Colab Notebooks/datasets/object_detection`

I created this folder inside of my `Colab Notebooks`, inside of `/datasets/object_detection` I manually uploaded  my `.tfrecords` and `label_map.pbtxt`

In [0]:
# Check the folder with the dataset into the Drive

%ls '/content/drive/My Drive/Colab Notebooks/datasets/object_detection'

pascal_label_map.pbtxt  train.tfrecord  val.tfrecord


## 4. Edit our pipeline.config

Inside of our downloaded model, there are the file `pipeline.config`, this must contain information about where is our tfrecords , label_map.pbtxt and pre-trained model last checkpoint.

Modify the 

In [0]:
import re

# Main path of where are we working on
object_detection_api_path = '/content/drive/My Drive/Colab Notebooks/object_detection/models/research/object_detection/'

MODEL_NAME = 'ssd_mobilenet_v2_quantized_300x300_coco_2019_01_03'

pretrained_path = object_detection_api_path + MODEL_NAME + '/model.ckpt'
pipeline_path = object_detection_api_path + MODEL_NAME + '/pipeline.config'

# Where is my tfrecords.
train_tfrecord_path = '/content/drive/My Drive/Colab Notebooks/datasets/object_detection/train.tfrecord'
val_tfrecord_path = '/content/drive/My Drive/Colab Notebooks/datasets/object_detection/val.tfrecord'
label_map_path = '/content/drive/My Drive/Colab Notebooks/datasets/object_detection/pascal_label_map.pbtxt'


# Inside of the pipeline.config replace the default paths with our paths and also
# num_classes num of examples
# There is a bug in the API if I set 'include_metrics_per_category: true', I change this
# to false.

with open(pipeline_path) as f:
  s = f.read()
with open(pipeline_path, 'w') as f:
  s = re.sub('PATH_TO_BE_CONFIGURED/model.ckpt', pretrained_path, s)
  s = re.sub('PATH_TO_BE_CONFIGURED/mscoco_train.record-00000-of-00100', train_tfrecord_path, s)
  s = re.sub('PATH_TO_BE_CONFIGURED/mscoco_val.record-00000-of-00010', val_tfrecord_path, s)
  s = re.sub('PATH_TO_BE_CONFIGURED/mscoco_label_map.pbtxt', label_map_path, s)
  s = re.sub('num_classes: 90', 'num_classes: 1', s)
  s = re.sub('num_examples: 8000', 'num_examples: 40', s)
  s = re.sub('include_metrics_per_category: true', 'include_metrics_per_category: false', s)

  # Lets see our final pipeline.config
  print(s)
  
  f.write(s)



model {
  ssd {
    num_classes: 1
    image_resizer {
      fixed_shape_resizer {
        height: 300
        width: 300
      }
    }
    feature_extractor {
      type: "ssd_mobilenet_v2"
      depth_multiplier: 1.0
      min_depth: 16
      conv_hyperparams {
        regularizer {
          l2_regularizer {
            weight: 3.99999989895e-05
          }
        }
        initializer {
          truncated_normal_initializer {
            mean: 0.0
            stddev: 0.0299999993294
          }
        }
        activation: RELU_6
        batch_norm {
          decay: 0.999700009823
          center: true
          scale: true
          epsilon: 0.0010000000475
          train: true
        }
      }
    }
    box_coder {
      faster_rcnn_box_coder {
        y_scale: 10.0
        x_scale: 10.0
        height_scale: 5.0
        width_scale: 5.0
      }
    }
    matcher {
      argmax_matcher {
        matched_threshold: 0.5
        unmatched_threshold: 0.5
        ignore_thresho

## 5. Training


*  Lets see our current folder, 
* Create our training folder inside of this folder
* Go one folder down to `research/` folder.
* Init tensorboard in background
* Training...



In [0]:

%pwd

'/content/drive/My Drive/Colab Notebooks/object_detection/models/research/object_detection'

In [0]:
# Create training folder into models/research/object_detection/

%mkdir training

mkdir: cannot create directory ‘training’: File exists


In [0]:
# go one folder down

% cd ..

/content/drive/My Drive/Colab Notebooks/object_detection/models/research


In [0]:
# Init tfboard in background

# Log tensorboard into the training/ folder
LOG_DIR = 'object_detection/training'

get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

In [0]:
# Download ngrok for expose the localhost to the internet.

!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

--2019-05-05 00:25:36--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 34.206.253.53, 54.174.228.92, 35.173.6.94, ...
Connecting to bin.equinox.io (bin.equinox.io)|34.206.253.53|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14991793 (14M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.3’

ngrok-stable-linux- 100%[===================>]  14.30M  52.8MB/s    in 0.3s    

2019-05-05 00:25:36 (52.8 MB/s) - ‘ngrok-stable-linux-amd64.zip.3’ saved [14991793/14991793]

Archive:  ngrok-stable-linux-amd64.zip
replace ngrok? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ngrok                   


In [0]:
# Init the ngrok service

get_ipython().system_raw('./ngrok http 6006 &')

In [0]:
# Get the exposed URL where I can access to the TensorBoard
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

http://616624f0.ngrok.io


--model_dir is the full path to the previous created `training ` folder.
--pipeline_config_path is the path where is my `pipeline.config`

In [0]:
# Training ..

!python object_detection/model_main.py \
  --model_dir="/content/drive/My Drive/Colab Notebooks/object_detection/models/research/object_detection/training" \
  --pipeline_config_path="/content/drive/My Drive/Colab Notebooks/object_detection/models/research/object_detection/ssd_mobilenet_v2_quantized_300x300_coco_2019_01_03/pipeline.config" \
  --alsologtostderr \
  --num_train_steps=15000 \
  --num_eval_steps=500


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use `tf.data.experimental.parallel_interleave(...)`.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
`seed2` arg is deprecated.Use sample_distorted_bounding_box_v2 instead.
Instructions for updating:
Use the `axis` argument instead
Instructions for updating:
Use `tf.data.Dataset.batch(..., drop_remainder=True)`.
2019-05-05 00:27:24.955388: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
2019-05-05 00:27:24.955663: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x103c47e0 executing computations on platform Host. Devices:
2019-05-05

## Links

https://github.com/tensorflow/models/pull/6585
https://github.com/tensorflow/models/pull/6585/commits/e0bb538c79f5e4a7b64d5c83fe6e08742eb015b4